Obective : To build a product recommender system for an ecommerce website

Data source : https://www.kaggle.com/carrie1/ecommerce-data/activity

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import random

from sklearn.metrics.pairwise import cosine_similarity

# import tensorflow as tf

In [2]:
df=pd.read_csv("/home/noname797/Work/ML_projects/Case_study/Datasets/recommender.csv")

In [3]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [4]:
print("df shape : ",df.shape)
print("Missing values : " ,df.isnull().sum().values.sum())

df shape :  (541909, 8)
Missing values :  136534


In [5]:
df.describe()


,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [6]:
df=df.loc[df["Quantity"]>0]
df=df.loc[df["UnitPrice"]>0]

In [7]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description         0
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     132220
Country             0
dtype: int64

In [8]:
df=df.dropna(subset=["CustomerID"])

In [9]:
df.shape

(397884, 8)

In [10]:
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [11]:
matrix=df.pivot_table(index="CustomerID",columns="StockCode",values="Quantity",aggfunc="sum")
matrix.head()

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12347.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12348.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0
12349.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
12350.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [12]:
matrix=matrix.applymap(lambda x: 1 if x>0 else 0)
matrix.head()

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12347.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12348.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
12349.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
12350.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [13]:
matrix.shape

(4338, 3665)

In [14]:
# Creating user to user similarity matrix using cosine rule

sim_matrix=pd.DataFrame(cosine_similarity(matrix))
sim_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,4328,4329,4330,4331,4332,4333,4334,4335,4336,4337
0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,0.0,1.000000,0.063022,0.046130,0.047795,0.038484,0.0,0.025876,0.136641,0.094742,...,0.0,0.029709,0.052668,0.0,0.032844,0.062318,0.0,0.113776,0.109364,0.012828
2,0.0,0.063022,1.000000,0.024953,0.051709,0.027756,0.0,0.027995,0.118262,0.146427,...,0.0,0.064282,0.113961,0.0,0.000000,0.000000,0.0,0.000000,0.170905,0.083269
3,0.0,0.046130,0.024953,1.000000,0.056773,0.137137,0.0,0.030737,0.032461,0.144692,...,0.0,0.105868,0.000000,0.0,0.039014,0.000000,0.0,0.067574,0.137124,0.030475
4,0.0,0.047795,0.051709,0.056773,1.000000,0.031575,0.0,0.000000,0.000000,0.033315,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.044866,0.000000


In [15]:
sim_matrix.columns=matrix.index
sim_matrix["CustomerID"]=matrix.index
sim_matrix=sim_matrix.set_index("CustomerID")
sim_matrix.head()

CustomerID,12346.0,12347.0,12348.0,12349.0,12350.0,12352.0,12353.0,12354.0,12355.0,12356.0,...,18273.0,18274.0,18276.0,18277.0,18278.0,18280.0,18281.0,18282.0,18283.0,18287.0
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
12347.0,0.0,1.000000,0.063022,0.046130,0.047795,0.038484,0.0,0.025876,0.136641,0.094742,...,0.0,0.029709,0.052668,0.0,0.032844,0.062318,0.0,0.113776,0.109364,0.012828
12348.0,0.0,0.063022,1.000000,0.024953,0.051709,0.027756,0.0,0.027995,0.118262,0.146427,...,0.0,0.064282,0.113961,0.0,0.000000,0.000000,0.0,0.000000,0.170905,0.083269
12349.0,0.0,0.046130,0.024953,1.000000,0.056773,0.137137,0.0,0.030737,0.032461,0.144692,...,0.0,0.105868,0.000000,0.0,0.039014,0.000000,0.0,0.067574,0.137124,0.030475
12350.0,0.0,0.047795,0.051709,0.056773,1.000000,0.031575,0.0,0.000000,0.000000,0.033315,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.044866,0.000000


In [16]:
i=random.choice(sim_matrix.index)
sim_matrix.loc[i].sort_values(ascending=False)

CustomerID
18080.0    1.000000
15049.0    0.408248
12873.0    0.353553
16698.0    0.353553
17991.0    0.353553
             ...   
14437.0    0.000000
14438.0    0.000000
14439.0    0.000000
14440.0    0.000000
18287.0    0.000000
Name: 18080.0, Length: 4338, dtype: float64

In [17]:
matrix.loc[i][matrix.loc[i]==1].index

Index(['22084', '84988'], dtype='object', name='StockCode')

In [18]:
matrix.loc[18064][matrix.loc[18064]==1].index

Index(['22084', '22090', '23152', '23202', '23203', '23208', '23209'], dtype='object', name='StockCode')

In [19]:
# What one bought and other didnot
rec_list=[i for i in matrix.loc[i][matrix.loc[i]==1].index if i not in matrix.loc[18064][matrix.loc[18064]==1].index]

In [20]:
df.loc[df["StockCode"].isin(rec_list),["StockCode","Description"]].drop_duplicates().set_index("StockCode")

,Description
StockCode,
84988,SET OF 72 PINK HEART PAPER DOILIES


In [21]:
item_matrix=pd.DataFrame(cosine_similarity(matrix.T))
item_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,3655,3656,3657,3658,3659,3660,3661,3662,3663,3664
0,1.000000,0.0,0.094868,0.091287,0.0,0.000000,0.090351,0.062932,0.098907,0.095346,...,0.0,0.0,0.0,0.0,0.0,0.029361,0.0,0.067591,0.0,0.078217
1,0.000000,1.0,0.000000,0.000000,0.0,0.000000,0.032774,0.045655,0.047836,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.016345,0.0,0.000000
2,0.094868,0.0,1.000000,0.115470,0.0,0.000000,0.057143,0.059702,0.041703,0.060302,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.071247,0.0,0.010993
3,0.091287,0.0,0.115470,1.000000,0.0,0.000000,0.164957,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
4,0.000000,0.0,0.000000,0.000000,1.0,0.447214,0.063888,0.044499,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000


In [22]:
item_matrix.columns=matrix.T.index
item_matrix["StockCode"]=matrix.T.index
item_matrix=item_matrix.set_index("StockCode")
item_matrix.head()

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
StockCode,,,,,,,,,,,,,,,,,,,,,
10002,1.000000,0.0,0.094868,0.091287,0.0,0.000000,0.090351,0.062932,0.098907,0.095346,...,0.0,0.0,0.0,0.0,0.0,0.029361,0.0,0.067591,0.0,0.078217
10080,0.000000,1.0,0.000000,0.000000,0.0,0.000000,0.032774,0.045655,0.047836,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.016345,0.0,0.000000
10120,0.094868,0.0,1.000000,0.115470,0.0,0.000000,0.057143,0.059702,0.041703,0.060302,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.071247,0.0,0.010993
10123C,0.091287,0.0,0.115470,1.000000,0.0,0.000000,0.164957,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
10124A,0.000000,0.0,0.000000,0.000000,1.0,0.447214,0.063888,0.044499,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000


In [23]:
i=random.choice(item_matrix.index)
item_matrix.loc[i].sort_values(ascending=False)

StockCode
17084P    1.000000
17084J    0.408248
17084N    0.404145
17084A    0.327327
21167     0.322749
            ...   
22524     0.000000
22525     0.000000
22526     0.000000
22529     0.000000
POST      0.000000
Name: 17084P, Length: 3665, dtype: float64

In [28]:
top_10_items=list(item_matrix.loc[i].sort_values(ascending=False).iloc[:10].index)

In [30]:
top_10_items

['17084P',
 '17084J',
 '17084N',
 '17084A',
 '21167',
 '17028J',
 '17001',
 '84795C',
 '35816P',
 '79336']

In [56]:
df.loc[df["StockCode"]==i][:1]["Description"].iloc[0]

'DRAGONS BLOOD INCENSE'

In [33]:
df.loc[df["StockCode"].isin(top_10_items),["StockCode","Description"]].drop_duplicates().set_index("StockCode").loc[top_10_items]

,Description
StockCode,
17084P,DRAGONS BLOOD INCENSE
17084J,LOVE POTION MASALA INCENSE
17084N,FAIRY DREAMS INCENSE
17084A,TRANQUILITY MASALA INCENSE
21167,WHITE SAGE INCENSE
17028J,INCENSE BAZAAR PEACH
17001,HEAVENS SCENT FRAGRANCE OILS ASSTD
84795C,OCEAN STRIPE HAMMOCK
35816P,"ACRYLIC JEWEL ANGEL,PINK"


In [61]:
def rec_items(item):
    top_10_list=list(item_matrix.loc[item].sort_values(ascending=False).iloc[:10].index)
    top_10=df.loc[df["StockCode"].isin(top_10_list),["StockCode","Description"]].drop_duplicates().set_index("StockCode").loc[top_10_list]
    print("Item : ",df.loc[df["StockCode"]==item][:1]["Description"].iloc[0])
    return top_10

In [62]:
rec_items("84029E")

Item :  RED WOOLLY HOTTIE WHITE HEART.


,Description
StockCode,
84029E,RED WOOLLY HOTTIE WHITE HEART.
84029G,KNITTED UNION FLAG HOT WATER BOTTLE
21479,WHITE SKULL HOT WATER BOTTLE
21485,RETROSPOT HEART HOT WATER BOTTLE
22111,SCOTTIE DOG HOT WATER BOTTLE
22112,CHOCOLATE HOT WATER BOTTLE
22114,HOT WATER BOTTLE TEA AND SYMPATHY
23355,HOT WATER BOTTLE KEEP CALM
84030E,ENGLISH ROSE HOT WATER BOTTLE
